在使用前，确保自己训练好了yolo模型，且下载了sam的官方预训练模型

In [ ]:
from ultralytics import YOLO
import numpy as np
from PIL import Image, ImageDraw
import torch
import torchvision
import sys
import cv2
import re
print(sys.path)
sys.path.append('your/path/to/segment-anything') #最好加一下，有的时候环境混乱会导致识别不到sam的路径
from segment_anything import SamPredictor, sam_model_registry #环境安装的sam库
import os
# 检查GPU可用性
print("GPU available: ", torch.cuda.is_available())
device='cuda:0' #指定GPU

In [ ]:
# 加载YOLO模型
checkpoint = "your/path/to/yolo/ckpt"  # YOLO训练好的模型路径
model = YOLO(checkpoint)
model.to(device)

In [ ]:
# 加载SAM模型
sam_checkpoint = "your/path/to/pretrained/sam/ckpt"  # 替换为你的SAM模型路径
sam_model = sam_model_registry["vit_h"](checkpoint=sam_checkpoint)  # 加载SAM的模型
sam_model.to(device)
sam_predictor = SamPredictor(sam_model)

In [ ]:
input_image = 'your/path/to/input/images or images dir'  # 输入图片路径,也可以是一个dir，yolo会自动处理，dir的话，sam那边需要自己处理一下
boxes=[]
names=["your/list/of/your/classes"] #你的类别名称.和数据集的yaml里的要一致

In [ ]:
# 获取YOLO检测的边界框
results = model(source=input_image, conf=0.2, show=False, save=True)
boxes = results[0].boxes.xyxy  # 获取边界框坐标 (xmin, ymin, xmax, ymax)
cls=results[0].boxes.cls
b=[]
for i in range(boxes.shape[0]):
    b.append((round(boxes[i][0].item(),1),round(boxes[i][2].item(),1),round(boxes[i][1].item(),1),round(boxes[i][3].item(),1),names[int(cls[i].item())]))
for i in b:
    print(i)

笔者保存了四种图片，假设原图为1.jpg
1. yolo检测的输出图 ，不需要时请将上一个cell的save改为 False，当save为true时，yolo会自动保存预测结果图，但不建议，会影响下一个cell的保存路径

下面三种后缀都是一定的，如不需要将下面cell中对应的保存代码删除即可

2. sam根据yolo检测框生成的gt_mask图      1_gtmask.png 

3. 根据gt_mask取出的部分原图             1_segmented.png

4. gt_mask和原图的重叠图                 1_masked.png

In [ ]:
def draw_yolo_style_box(image, xmin, ymin, xmax, ymax, cls, color=(255, 0, 0)):
    # 绘制边框
    cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 2)

In [ ]:
allow=["your/list/of/your/classes that you want to segment"] #你想要分割的类别名称
# 假设你已经加载了图像并且boxes是YOLO检测到的边界框（xmin, ymin, xmax, ymax）
image =cv2.imread(input_image)
image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
output_image = np.copy(image)
file_name = "your/outpout/dir/name"    #根据需求自行修改
image_with_boxes=image.copy()

In [ ]:
# 创建输出目录，简单来说直接删除代码只剩最后两行，直接指定就行
pathout = 'your/path/to/runs/detect'  #这个路径是上面save为true时yolo自动创建的保存结果路径

# 获取目录下的所有文件夹
folders = os.listdir(pathout)

# 正则表达式匹配 'expX' 中的数字
exp_numbers = []

# 遍历文件夹并提取 'exp' 文件夹中的数字
for folder in folders:
    match = re.match(r'predict(\d+)', folder)
    if match:
        exp_numbers.append(int(match.group(1)))

# 获取最大的 i (expX 中的最大数字)
if exp_numbers:
    max_exp = max(exp_numbers)
    print(f"The largest 'i' is: {max_exp}")
else:
    print("No 'expX' folders found.")

# 输出文件夹
output_dir = f'./runs/detect/predict{max_exp}/{file_name}'
print(output_dir)

In [ ]:
os.makedirs(output_dir, exist_ok=True)
grid = [] 
grid.append(Image.fromarray(image))
# 创建一个空的掩码图像（和原图一样的大小）
combined_mask = np.zeros_like(image[:, :, 0])  # 创建与原图大小相同的空白掩码图（灰度图）
sam_predictor.set_image(image)
# 对每个检测框进行分割并将掩码叠加到合成掩码图上
for i, boxi in enumerate(b):
    xmin, xmax, ymin, ymax,cls= boxi
    print(boxi)
    if cls in allow:# 保证只对需要的cls进行分割，减少开销
        
        draw_yolo_style_box(image_with_boxes, xmin, ymin, xmax, ymax, cls, color=(255, 0, 0))
        box = np.array([xmin,ymin,xmax,ymax])
        # 使用SAM模型直接通过边界框进行分割
        mask, _, _ = sam_predictor.predict(
            point_coords=None,
            point_labels=None,
            box=box,  # 直接将整个边界框传入进行分割
            multimask_output=False
        )
        
        # 遍历每个掩码（如果有多个掩码）
        for m in mask:
            # 将掩码从 [height, width] 转换为 [height, width, 1] 的形状，适应整个图像
            m_resized = m[:, :, None]
            m_resized = np.broadcast_to(m_resized, image.shape)  # 将掩码广播到整个图像形状
            
            # 根据掩码将图像的对应区域修改为指定的值（例如 0）
            modified_part = np.where(m_resized == 1, 30, image)
            grid.append(Image.fromarray(modified_part))
            # 将掩码区域合并到合成掩码图中
            combined_mask = np.maximum(combined_mask, m_resized[:, :, 0] * 255)
            
            # 保存图像部分的修改结果（如果需要逐个保存每个掩码图）
            grid.append(Image.fromarray(modified_part))

# 保存合成掩码图
mask_output_path = os.path.join(output_dir, f"{file_name}_gtmask.png")
combined_mask_pil = Image.fromarray(combined_mask.astype(np.uint8))
#combined_mask_pil.save(mask_output_path)
print(f"Saved the combined mask image to {mask_output_path}")

# 将合成掩码图与原图重合

output_image_with_mask = np.where(combined_mask[:, :, np.newaxis] == 255, 150 ,output_image)  # 将掩码区域保留，其他区域设置为黑色
output_image_only_mask = np.where(combined_mask[:, :, np.newaxis] == 255 ,output_image, 150)

# 保存最终图像（合成图与掩码重合的图）
output_path = os.path.join(output_dir, f"{file_name}_masked.png")
output_image_with_mask_pil = Image.fromarray(output_image_with_mask.astype(np.uint8))
#output_image_with_mask_pil.save(output_path)

output_path2= os.path.join(output_dir, f"{file_name}_segmented.png")
output_image_only_mask_pil = Image.fromarray(output_image_only_mask.astype(np.uint8))
#output_image_only_mask_pil.save(output_path2)
print(f"Saved the combined segmented image with masks to {output_path}")
print("Done!")
# 保存画框图像
detect_output_path = os.path.join(output_dir, f"{file_name}_detect.jpg")
Image.fromarray(image_with_boxes.astype(np.uint8)).save(detect_output_path)
print(f"Saved the detection box image to {detect_output_path}")

# 设置黄色和透明度
highlight_color = np.array([255, 255, 0], dtype=np.uint8)  # 黄色
alpha = 0.3  # 透明度系数，0=完全原图，1=完全高亮色

# 复制原图
blended_image = output_image.copy().astype(np.float32)

# 找掩码区域
mask_region = combined_mask == 255

# 将掩码区域的像素值进行 alpha 混合：output = original * (1 - alpha) + highlight * alpha
for c in range(3):  # 对 R,G,B 三个通道分别处理
    blended_image[:, :, c][mask_region] = (
        (1 - alpha) * blended_image[:, :, c][mask_region] + alpha * highlight_color[c]
    )

# 保存图像
output_path = os.path.join(output_dir, f"{file_name}_highlighted_mask.png")
highlighted_pil = Image.fromarray(blended_image.astype(np.uint8))
highlighted_pil.save(output_path)